In [2]:
import pandas as pd
from datetime import datetime, timedelta

# Ruta del archivo de entrada CSV
input_csv_path = 'D:\\Trufiapp\\GANS\\forecast\\forcast_data.csv'
id_index_h3 = 'D:\\Trufiapp\\GANS\\id_index_h3.csv'
id_index_h3 = pd.read_csv(id_index_h3)
# Ruta del archivo de salida CSV
output_csv_path = 'D:\\Trufiapp\\GANS\\forecast\\forcast_data_ready.csv'

# Leer el CSV original como DataFrame
df = pd.read_csv(input_csv_path, delimiter=',')
# df.reset_index(inplace=True)
# Convertir la columna 'id' a tipo int64
id_index_h3['id'] = id_index_h3['id'].astype('str')
id_index_h3.describe()

,latitude,longitude
count,497.000000,497.000000
mean,-17.450013,-66.073500
std,0.166469,0.289389
min,-18.192599,-66.762860
25%,-17.561841,-66.261813
50%,-17.442331,-66.109412
75%,-17.340737,-65.940860
max,-16.607410,-64.589550


In [3]:
# Melt para convertir las columnas adicionales
df_long = pd.melt(df, id_vars=['hourly_timestamp'], var_name='id', value_name='quantity')

# Convertir el formato de la columna 'hourly_timestamp'
df_long['hourly_timestamp'] = pd.to_datetime(df_long['hourly_timestamp'], format='%Y-%m-%d %H:%M:%S')

# Ordenar el DataFrame por la columna 'hourly_timestamp' y 'id'
df_long.sort_values(by=['hourly_timestamp'], inplace=True)

# Convertir la columna 'id' a tipo int64
df_long['id'] = df_long['id'].astype(str)
# df_long.reset_index(inplace=True)
df_long.describe()

,hourly_timestamp,quantity
count,9912,9912.000000
mean,2023-12-19 11:30:00.000000256,1.096868
min,2023-12-16 00:00:00,-0.102179
25%,2023-12-17 17:45:00,0.001798
50%,2023-12-19 11:30:00,0.034883
75%,2023-12-21 05:15:00,0.370706
max,2023-12-22 23:00:00,69.307882
std,NaN,3.797623


In [4]:

# Asegurarse de que la columna 'id' en df_long sea de tipo str
df_long['id'] = df_long['id'].astype(str)
# Eliminar filas donde la columna 'id' sea igual a 'index'
df_long = df_long[df_long['id'] != 'index']

# Fusión (merge) para reemplazar 'id' con 'h3_index' en df_long
df_long = pd.merge(df_long, id_index_h3[['id', 'h3_index']], how='left', on='id')

# Eliminar la columna 'id' ya que ha sido reemplazada por 'h3_index'
df_long.drop(columns=['id'], inplace=True)

# Renombrar la columna 'h3_index' a 'id'
df_long.rename(columns={'h3_index': 'id'}, inplace=True)
df_long

,hourly_timestamp,quantity,id
0,2023-12-16 00:00:00,2.882665,878b2c8a1ffffff
1,2023-12-16 00:00:00,-0.004188,878b2c836ffffff
2,2023-12-16 00:00:00,0.032169,878b2c8b5ffffff
3,2023-12-16 00:00:00,0.074659,878b2c988ffffff
4,2023-12-16 00:00:00,1.549848,878b2c8a8ffffff
...,...,...,...
9907,2023-12-22 23:00:00,0.252786,878b2c8a2ffffff
9908,2023-12-22 23:00:00,-0.000356,878b2c983ffffff
9909,2023-12-22 23:00:00,0.147856,878b2c99dffffff
9910,2023-12-22 23:00:00,0.043869,878b2c991ffffff


In [5]:
df_long.to_csv(output_csv_path, index=False)